# Here the algorithm of **regret matching** for Rock-Paper-Scissors to find the best response strategy to the strategy of the opponent is released

In [1]:
import random
random.seed(0)

Define constants:

In [2]:
ROCK, PAPER, SCISSORS = 0, 1, 2
NUM_ACTIONS = 3

# Compute action utilities using utility matrix
# myAction - choose row, otherAction - choose col, RPS accordingly to the order
actionUtility = [[0, -1, 1],
                    [1, 0, -1],
                    [-1, 1, 0]]

regretSum = [0] * NUM_ACTIONS
strategy = [0] * NUM_ACTIONS
strategySum = [0] * NUM_ACTIONS

# Choose a strategy for the opponent
oppStrategy = [0.4, 0.3, 0.3]

Function to get current mixed strategy through regret-matching:

In [3]:
def get_strategy():
    normalizing_sum = 0
    for action in range(NUM_ACTIONS):
        strategy[action] = max(0, regretSum[action])
        normalizing_sum += strategy[action]

    for action in range(NUM_ACTIONS):
        if normalizing_sum > 0:
            strategy[action] /= normalizing_sum
        else:
            # If we don't regret about anything
            strategy[action] = 1.0 / NUM_ACTIONS
        strategySum[action] += strategy[action]

    return strategy

Function to get random action according to mixed-strategy distribution:

In [4]:
def get_action(strategy):
    # Next time for this function numpy will be used
    r = random.random()
    action = 0
    cumulativeProbability = 0
    while action < NUM_ACTIONS - 1:
        cumulativeProbability += strategy[action]
        if r < cumulativeProbability:
            break
        action += 1
    return action


Function to get average mixed strategy across all training iterations

In [5]:
def get_average_strategy():
    avg_strategy = [0.0] * NUM_ACTIONS
    normalizing_sum = sum(strategySum)
    for action in range(NUM_ACTIONS):
        if normalizing_sum > 0:
            avg_strategy[action] = strategySum[action] / normalizing_sum
        else:
            avg_strategy[action] = 1.0 / NUM_ACTIONS
    return avg_strategy


With these building blocks in place, we can now construct our training algorithm:

In [6]:
def train(iterations):
    for _ in range(iterations):

        # Get regret-matched mixed strategy actions
        strategy = get_strategy()
        myAction = get_action(strategy)
        otherAction = get_action(oppStrategy)

        # Accumulate action regrets
        for action in range(NUM_ACTIONS):
            # Case if we consider negative regrets
            regretSum[action] += (actionUtility[action][otherAction] - actionUtility[myAction][otherAction])

            # Case if we don't consider negative regrets
            # regretSum[action] += max(0 , actionUtility[action][otherAction] - actionUtility[myAction][otherAction])

Now we are ready to run the computation of the response strategy:

In [7]:
# Obtained strategy converges to [0, 1, 0] if we consider negative regrets
# And to ~ [0.33, 0,36, 0,31] if we don't
train(iterations=100_000)
print(get_average_strategy())

[0.00022815968170315993, 0.9993322188194254, 0.0004396214988714989]


In [8]:
# Comparison of these two cases

strategyPN = [0.0022815968170315994, 0.9933221881942534, 0.004396214988714989]
strategyP = [0.32656612190646006, 0.36233503087561797, 0.31109884721792197]

expUtilityPN = 0
expUtilityP = 0

for i in range(NUM_ACTIONS):
    for j in range(NUM_ACTIONS):
        expUtilityPN += strategyPN[i] * oppStrategy[j] * actionUtility[i][j]
        expUtilityP += strategyP[i] * oppStrategy[j] * actionUtility[i][j]
        
print(f' Expected utility if we consider positive and negative regrets: {expUtilityPN:0.3f}')
print(f' Expected utility if we consider only positive regrets: {expUtilityP:0.3f}')

 Expected utility if we consider positive and negative regrets: 0.099
 Expected utility if we consider only positive regrets: 0.005


So we can conclude that it is better to consider wins and losses for regret minimization, because the expected utility will be higher